In [ ]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

URL = "https://search.shopping.naver.com/search/all?query=%EB%8B%AD%EA%B0%80%EC%8A%B4%EC%82%B4"


# 크롬 드라이버 생성
driver = webdriver.Chrome()

# 페이지 이동
driver.get(URL)

In [5]:
#무한스크롤

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    #잠시 대기(컴퓨터에게 로딩시간을 줘야함)
    time.sleep(2)

    # 스크롤 후 높이
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 비교(if, breack)
    if new_height == last_height:
        break

    # 스크롤 전 높이 업데이트
    last_height = new_height

In [ ]:
import pandas as pd 

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#나무 선택자 찾기

data = []
items = soup.select(".product_item__MDtDF")
for item in items:
    name  = item.select_one(".product_title__Mmw2K > a").text
    link = item.select_one(".product_title__Mmw2K > a").attrs['href']
    price = item.select_one(".price_num__S2p_v").text.replace("원" , "").replace("," , "")
    
    # 속성선택자
    if item.select_one("[data-shp-area*=purchase] em"):
        purchase = item.select_one("[data-shp-area*=purchase] em").text
        
        # 엑셀에 뿌리니깐 숫자가 뒤죽박죽이여서 스스로 한번 조건을 달고 정리해봄
        # 결과는 대성공~
        if '만' in purchase:
        
            purchase = int(float(purchase.split('만')[0]) * 10000)

        elif len(purchase) <= 3:
            purchase = int(purchase)

        elif ',' in purchase:
            purchase = int(purchase.replace("," , ""))
       
    else:
        purchase = ""


    data.append([name, link, price, purchase])

df = pd.DataFrame(data, columns= ["상품명" ,"링크", "가격", "구매건수"])  

df.to_excel("naverShop.xlsx") 